In [86]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
import scipy.integrate as integrate
import glob
import os


from ipywidgets import VBox,HBox, interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.layouts import column, row, gridplot
from bokeh.models import BoxAnnotation, DataTable, TableColumn, PointDrawTool, HTMLTemplateFormatter

#from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter

from src.spectrum import spectrum
from src.spec_plot import spec_plot
from src.lcurve_plot import lcurve_plot
from src.mycolors import mycolors
from src.background_models import bg3D

output_notebook()

filelist = [os.path.basename(file) for file in glob.glob(os.path.join('data','raw','*.DTA'))]
file_picker=widgets.Dropdown(description= 'Choose File:', options= filelist)
file_picker

Loading BokehJS ...

Dropdown(description='Choose File:', options=('example_2dDEER.DTA',), value='example_2dDEER.DTA')

In [2]:
spec = spectrum(os.path.join('data','raw',file_picker.value))


#Plot definitions
xs= [spec.raw_time]*2
ys = [spec.real2D[0],spec.imaginary2D[0]]

spec2D = spec_plot(xs,ys,y_range = (spec.imaginary2D.min(),spec.real2D.max()*1.02), plot_height=300)

ys = [spec.real,spec.imaginary]
    
specavg = spec_plot(xs,ys, y_range = (spec.imaginary.min(),spec.real.max()*1.02), plot_height=300) 


#Widget Defintions

def on_button_clicked(val):
    if np.sum(spec.slice_mask)>1: #prevent deleting of all slices
        spec.delete_slice(myslider.value)
        myslider.options=valuelist[spec.slice_mask]
        myslider.value = myslider.options[-1]
        spec.avg_slices()
        update(myslider.value)
    
def update(index):
    for idx,line in enumerate(spec2D.renderers):
        line.data_source.data['y'] = [spec.real2D,spec.imaginary2D][idx][index]
        
    for idx,line in enumerate(specavg.renderers):
        line.data_source.data['y'] = [spec.real,spec.imaginary][idx]
        
    push_notebook(handle=sliceplots)
    return index

valuelist=np.arange(spec.real2D.shape[0])

myslider=widgets.SelectionSlider(description='value', value=valuelist[-1],options=valuelist)
deletebutton=widgets.Button(description='Delete Slice')
deletebutton.on_click(on_button_clicked)



#Render plots, widgets
sliceplots=show(row(spec2D,specavg),notebook_handle=True)
HBox([interactive(update,index=myslider),deletebutton])


In [3]:
cutoffx=[spec.raw_time.max(),spec.raw_time.max()]
cutoffy=[-.1,1]

cutoffslider=widgets.SelectionSlider(description='Cutoff', value=spec.raw_time[-1], options=spec.raw_time)
signal=spec.real+1j*spec.imaginary
forphasing=signal[int(len(signal)/8):list(spec.raw_time).index(cutoffslider.value)]
spec.phase=optimize.fmin(lambda x:abs((forphasing*np.e**(1j*x)).imag.mean()),1,disp=False)[0]
phaseslider=widgets.FloatSlider(description='Phase', value=spec.phase, min=-np.pi, max=np.pi, step=0.001)


xs = [spec.raw_time]*2
ys = [np.real(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase))),
      np.imag(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))]


phaseplot=spec_plot(xs,ys)
phaseplot.line(cutoffx,cutoffy, color=(255,0,0), line_width=1) #decorate plot with cutoff line
phaser=show(phaseplot, notebook_handle=True);


phasebutton=widgets.Button(description='Phase')
kernelbutton=widgets.Button(description='Build Kernel')

def phase_button_clicked(val):
    forphasing=signal[int(len(signal)/8):list(spec.raw_time).index(cutoffslider.value)]
    spec.phase=optimize.fmin(lambda x:abs((forphasing*np.e**(1j*x)).imag.mean()),1,disp=False)[0]
    phaseslider.value=spec.phase
    
def kernel_button_clicked(val):
    
    spec.tmax = cutoffslider.value
    spec.cutoff=cutoffslider.value # is cutoff needed (and not just as spec.tmax) ??
    spec.build_kernel()
    spec.real=np.real(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
    spec.imag=np.imag(signal*np.e**(1j*spec.phase))/np.max(np.real(signal*np.e**(1j*spec.phase)))
    
   

    
def updatePhase(value):
    phaseplot.renderers[-3].data_source.data['y'] = np.real(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))
    phaseplot.renderers[-2].data_source.data['y'] = np.imag(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))
    phaseplot.renderers[-1].data_source.data['y'] = [np.min(np.imag(signal*np.e**(1j*value))/np.max(np.real(signal*np.e**(1j*value)))),1]
    push_notebook(handle=phaser)
    return value

def updateCutoff(value):
    phaseplot.renderers[-1].data_source.data['x'] = [value,value]
    push_notebook(handle=phaser)
    return value

#kernel,spec.t,spec.r,Lmatrix,spec.cutoff = 

phasebutton.on_click(phase_button_clicked)
kernelbutton.on_click(kernel_button_clicked)
HBox([interactive(updateCutoff,value=cutoffslider),interactive(updatePhase,value=phaseslider),phasebutton,kernelbutton])

In [4]:
#initial fit, finding spectrum start
#need to reimplement different types of background models

def gaussian(x,r,s,a):
    return a*np.exp(-((x-r)**2/(2*s**2)))

symmetry_point=np.argmax(spec.raw_time>=-spec.raw_time[0])+1 #find index of symmetry point about t=0,+1 for non inclusive
spec.zeropoint,__,spec.zeroamp = gaussfit = optimize.curve_fit(gaussian,
                                                               spec.raw_time[1:symmetry_point],
                                                               spec.real[1:symmetry_point])[0]
spec.update_ranges()
spec.fit_background(bg3D,spec.background_range)
spec.background_correct()

#zeropeak plot

xs = [spec.raw_time[1:symmetry_point]]*2
ys = [spec.real[1:symmetry_point],
      gaussian(spec.raw_time,*gaussfit)[1:symmetry_point]]

zeropointx = [spec.zeropoint]*2
zeropointy = [spec.real[1:symmetry_point].min(),1]

zeropeak = spec_plot(xs,ys, plot_height=300)
zeropeak.line(zeropointx, zeropointy, color=(0,255,0))
zeropeak.circle([spec.zeropoint], [spec.zeroamp], color=(255,0,0))

#bg fit plot

xs = [spec.raw_time]*2
ys = [spec.real,spec.background]

cutoffx=[spec.cutoff]*2
cutoffy=[spec.real.min(),1]

backgroundx=[spec.background_start]*2
backgroundy=[spec.real.min(),1]

bgsub=spec_plot(xs,ys,plot_height=300)
bgsub.line(cutoffx,cutoffy,color=(255,0,0))
bgsub.line(backgroundx,backgroundy,color=(0,0,255))

#waveform plot 
waveformplot=spec_plot(spec.t,spec.waveform,plot_height=300)


processingplots = show(gridplot([[zeropeak,bgsub],[None,waveformplot]]),notebook_handle=True)

windowslider=widgets.IntSlider(description='Fit window', 
                               value=symmetry_point, min=4+np.argmax(spec.real==spec.real.max()), max=round(len(spec.raw_time)/30))
bgslider=widgets.SelectionSlider(description='Background Start', 
                                 value=spec.background_start, 
                                 options=spec.raw_time[4+np.argmax(spec.real==spec.real.max()):np.argmax(spec.raw_time==spec.cutoff)-20])
normalizer=widgets.Dropdown(description= 'V(t=0)', options=["Gaussian Fit",
                                   "Gaussian Center, Data Maximum Amp", 
                                   "Data Max Center, Data Maximum Amp"])

def updateBg(value): 
    spec.background_start = value
    spec.update_ranges()

    spec.fit_background(bg3D,spec.background_range)
    bgsub.renderers[-3].data_source.data['y'] = spec.background
    bgsub.renderers[-1].data_source.data['x']=[value]*2
    
    spec.background_correct()
    
    waveformplot.renderers[-1].data_source.data={'x':spec.t,'y':spec.waveform}
    
    push_notebook(handle=processingplots)


def updateWindow(value):
    spec.zeropoint,__,spec.zeroamp = gaussfit = optimize.curve_fit(gaussian,
                                                                   spec.raw_time[1:value],
                                                                   spec.real[1:value])[0]

    if normalizer.value == "Gaussian Center, Data Maximum Amp":
        spec.zeroamp = spec.real[abs(spec.raw_time-spec.zeropoint).argmin()]
    elif normalizer.value == "Data Max Center, Data Maximum Amp":
        spec.zeropoint = spec.raw_time[np.argmax(spec.real==spec.real.max())]
        spec.zeroamp = spec.real.max()
        
    spec.update_ranges()
    updateBg(bgslider.value)
    
    newx=spec.raw_time[1:value]
    newy=spec.real[1:value]
    fity=gaussian(newx,*gaussfit)
    plot_order = [[newx,newy],
                  [newx,fity],
                  [[spec.zeropoint]*2,[newy.min(),1]],
                  [[spec.zeropoint],[spec.zeroamp]]]
    
    for idx, line in enumerate(zeropeak.renderers):
        line.data_source.data = {'x': plot_order[idx][0], 'y': plot_order[idx][1]}

    
    push_notebook(handle=processingplots)
    

def updateNorm(value):
    normMethod = value
    updateWindow(windowslider.value)
    updateBg(bgslider.value)

VBox([HBox([interactive(updateWindow,value=windowslider),interactive(updateNorm,value=normalizer)]),
HBox([interactive(updateBg,value=bgslider)])])

In [5]:
spec.generate_lcurve()

#l-curve compoenents, initialize choice of alpha with LOOCV
residualnorm=np.linalg.norm(spec.tikhonovfits-spec.waveform,axis=1)**2
solutionnorm=np.linalg.norm(np.dot(spec.Lmatrix,spec.solutions.T).T,axis=1)**2
alpha_LOOCV_index=(residualnorm/(np.sum(1-spec.hMats,axis=1))**2).argmin()

#NOT SURE IM COMPUTING LOOCV CORRECTLY ABOVE, NEED TO GO RE-WORK


#l-curve
lcurve = lcurve_plot([residualnorm,solutionnorm],plot_height=300)
lcurve.circle([residualnorm[alpha_LOOCV_index]], [solutionnorm[alpha_LOOCV_index]],color=(255,0,0)) #LOOCV choice for alpha
lcurve.circle([residualnorm[alpha_LOOCV_index]], [solutionnorm[alpha_LOOCV_index]],color=(0,0,0)) #user defined choice for alpha

#waveform fit
xs = [spec.t]*2
ys = [spec.waveform,spec.tikhonovfits[alpha_LOOCV_index]]

tikresults=spec_plot(xs,ys,plot_height=300)

#fit residual
tikresidual=spec_plot(spec.t,spec.waveform-spec.tikhonovfits[alpha_LOOCV_index],
                      plot_height=300,
                      y_range=([(spec.waveform-spec.tikhonovfits[alpha_LOOCV_index]).min(),
                                (spec.waveform-spec.tikhonovfits[alpha_LOOCV_index]).max()]))

#Distance distributions

xs = [spec.r]*2
ys = [spec.solutions[alpha_LOOCV_index]]*2
distances = spec_plot(xs,ys,plot_height = 300,colorlist = [(255,0,0),mycolors(0)])
distances.xaxis.axis_label="Distance (nm)"
# show the results


lcurveplots = show(gridplot([[lcurve,tikresidual],[tikresults,distances]]),notebook_handle=True)

def updateAlpha(value):
    spec.alpha=spec.alphas[value]
    tikresidual.renderers[-1].data_source.data['y']=spec.waveform-spec.tikhonovfits[value]
    tikresults.renderers[-1].data_source.data['y']=spec.tikhonovfits[value]
    lcurve.renderers[-1].data_source.data={'x':[residualnorm[value]],'y':[solutionnorm[value]]}
    distances.renderers[-1].data_source.data['y'] = spec.solutions[value]
    spec.solution = spec.solutions[value]
    spec.tikhonovfit = spec.tikhonovfits[value]

    push_notebook(handle=lcurveplots)

#slider for index of list of alphas, actual alpha value can be found with spec.alpha
alphaslider=widgets.IntSlider(description='Alpha Value', value=alpha_LOOCV_index, min=0, max=len(spec.alphas)-1)
HBox([interactive(updateAlpha,value=alphaslider)])

In [6]:
##BG VALIDATION

spec.validate_background()

from scipy.signal import argrelextrema
from scipy.stats import f as ftest

# Measure residual of fits rescaled to uncorrected bg spectra to normalize noise

amplitudes = [(1.0 - spec.backgrounds[i,spec.waveform_range][0])/(spec.backgrounds[i,spec.waveform_range][0]) 
              for i in range(len(spec.backgrounds))]

reconstructed = [(np.dot(spec.kernel,spec.validation_solutions[i])/np.dot(spec.kernel,spec.validation_solutions[i]).max())
                 *amplitudes[i]*spec.backgrounds[i,spec.waveform_range]+spec.backgrounds[i,spec.waveform_range] 
                 for i in range(len(spec.backgrounds))]

validation_RMSD = np.sqrt(np.mean((spec.real[spec.waveform_range] - reconstructed)**2, axis = 1))

# find local maxima, wellR and find wellL at the same RMSD value
maxima_index = argrelextrema(validation_RMSD, np.greater)[0][0]


wellL = np.where(validation_RMSD[:maxima_index]>=validation_RMSD[maxima_index])[0][-1]
wellR = maxima_index
RMSD_min_index = validation_RMSD[wellL:wellR].argmin()


xs = spec.raw_time[[np.argwhere(spec.raw_time == point)[0][0] for point in spec.background_fit_points]]
ys = validation_RMSD
rmsd_plot = spec_plot(xs,ys,plot_height = 300, y_range = (min(validation_RMSD[wellL:wellR])*.98,validation_RMSD[wellL]))

#my_DOF this is clearly incorrect, need to figure out DOF based on LOOCV but is good enought
#since the purpose is to determine the bottoms of the background validation well only.
my_DOF = len(spec.waveform) - 3 
# f_test = np.where(0.6 >= np.round(ftest.cdf(validation_RMSD[wellL:wellR]/np.median(validation_RMSD[wellL:wellR]),my_DOF,my_DOF),decimals = 2))[0]
f_test = np.where(0.6 >= np.round(ftest.cdf(validation_RMSD/validation_RMSD[RMSD_min_index],my_DOF,my_DOF),decimals = 2))[0]
f_test = f_test[:np.where(1<(f_test[1:]-f_test[:-1]))[0][0]+1] #find first discontinuity in ftest (edge of well)
rmsd_plot.circle(xs,ys)

rmsd_plot.line(xs[f_test],validation_RMSD[f_test],color = (0,0,0), line_width=3)
rmsd_plot.circle([xs[RMSD_min_index]],[validation_RMSD[RMSD_min_index]], color = (255,0,0))
rmsd_plot.circle([xs[RMSD_min_index]],[validation_RMSD[RMSD_min_index]], color = (0,255,0))

rmsd_plot.xaxis.axis_label="Background Fit Start (µs)"
rmsd_plot.yaxis.axis_label_text_font='helvetica'
rmsd_plot.yaxis.axis_label_text_font_size='14pt'


rmsd_plot.yaxis.axis_label="Spectrum Fit RMSD"

backgrounds_plot=spec_plot([spec.raw_time]*3, #x-values
                      [spec.real,spec.background,spec.backgrounds[RMSD_min_index]], #y-values
                      plot_height=300)

spectrums_plot = spec_plot([spec.t]*2,
                                 [spec.waveforms[RMSD_min_index],spec.validation_tikhonovfits[RMSD_min_index]],
                                 plot_height = 300, y_range = (np.min(spec.waveforms),1.1)) #This is a DDT Lab joke
spectrums_plot.line(spec.t,0,line_color = 'black', line_dash = 'dashed', line_width = 1)

distances_plot = spec_plot([spec.r]*2,
                                   [spec.validation_solutions[RMSD_min_index],spec.validation_solutions[RMSD_min_index]],
                                   y_range = (0,np.max(spec.validation_solutions)), 
                                   plot_height = 300)
distances_plot.varea(x=spec.r,
        y1=np.quantile(spec.validation_solutions,0, axis=0),
        y2=np.quantile(spec.validation_solutions,1, axis=0), fill_alpha = 0.2, level = 'underlay')

distances_plot.xaxis.axis_label="Distance (nm)"


validation_plots = show(gridplot([[rmsd_plot,backgrounds_plot],[spectrums_plot,distances_plot]]), notebook_handle = True)

def updateRegion(end_points):
    new_range = range(*end_points)
    rmsd_plot.renderers[-3].data_source.data={'x':xs[new_range],'y': validation_RMSD[new_range]}
    try:
        distances_plot.renderers[-1].data_source.data = {'x': spec.r,
                                                         'y1': np.quantile(spec.validation_solutions[new_range], 0, axis=0),
                                                         'y2': np.quantile(spec.validation_solutions[new_range], 1, axis=0)}
        bgslider.options = new_range
    except IndexError:
        print('Zero width range, distances envelope not updated')
        bgslider.options = [end_points[0]]
        #dont update if range is 0 width
    
    push_notebook(handle = validation_plots)
    
def updateSelection(value):
    rmsd_plot.renderers[-1].data_source.data={'x':[xs[value]],'y': [validation_RMSD[value]]}
    backgrounds_plot.renderers[-1].data_source.data['y'] = spec.backgrounds[value]
    spectrums_plot.renderers[-3].data_source.data['y'] = spec.waveforms[value]
    spectrums_plot.renderers[-2].data_source.data['y'] = spec.validation_tikhonovfits[value]
    distances_plot.renderers[-2].data_source.data['y'] = spec.validation_solutions[value]
    
    spec.background_start = spec.background_fit_points[bgslider.value]
    spec.waveform = spec.waveforms[bgslider.value]
    spec.tikhonovfit = spec.validation_tikhonovfits[bgslider.value]
    spec.solution = spec.validation_solutions[bgslider.value]
    
    push_notebook(handle = validation_plots)

rmsdSlider = widgets.IntRangeSlider(description = 'Backgrounds:', value =list(f_test[[1,-1]]), min = 0, max = validation_RMSD.shape[0])
bgslider = widgets.SelectionSlider(description = 'Selection:', value = RMSD_min_index, options = range(rmsdSlider.value[0],rmsdSlider.value[1]+1))

VBox([interactive(updateRegion,end_points=rmsdSlider),interactive(updateSelection,value=bgslider)])




In [87]:
width_cutoff = 4*(spec.t[-1]/2)**(1/3)
distance_cutoff = 5*(spec.t[-1]/2)**(1/3)
#Jeschke, G. (2012). DEER distance measurements on proteins. Annual Review of Physical Chemistry, 63, 419–446.
instability_point_index = abs(spec.r-(width_cutoff+(distance_cutoff-width_cutoff)/2)).argmin()
instability_point_start = spec.r[instability_point_index]
distance_mask = spec.r*0
distance_mask[instability_point_index:] = 1

spec.unstable_waveform = np.dot(spec.kernel, spec.solution*distance_mask)
spec.stable_waveform = (spec.waveform - spec.unstable_waveform)/(1.0 - spec.unstable_waveform.max())
spec.stable_solution = spec.waveform_regularize(spec.stable_waveform)
spec.stable_solution = spec.stable_solution/integrate.trapz(spec.stable_solution)



instability_plot = spec_plot(spec.r,spec.solution,y_range = (0,spec.solution.max()*1.1), plot_height = 300)
instability_plot.line([instability_point_start]*2,[0,spec.solution.max()*1.1],color = (0,0,0), line_dash = 'dashed')
instability_plot.varea(x=spec.r,
                       y1=np.quantile(spec.validation_solutions[range(*rmsdSlider.value)],0, axis=0),
                       y2=np.quantile(spec.validation_solutions[range(*rmsdSlider.value)],1, axis=0), 
                       fill_alpha = 0.2, level = 'underlay')

green_box = BoxAnnotation(left=0, right=width_cutoff, fill_color='green', fill_alpha=0.1)
yellow_box = BoxAnnotation(left=width_cutoff, right=distance_cutoff, fill_color='yellow', fill_alpha=0.1)
red_box = BoxAnnotation(left=width_cutoff, right=10, fill_color='red', fill_alpha=0.1)
instability_plot.add_layout(green_box)
instability_plot.add_layout(yellow_box)
instability_plot.add_layout(red_box)
instability_plot.xaxis.axis_label = "Distance (nm)"

filtered_distances = spec_plot([spec.r,spec.r],
                               [spec.solution/integrate.trapz(spec.solution-spec.solution*distance_mask),spec.stable_solution],y_range = (-.001,spec.solution.max()*1.1), plot_height = 300)
filtered_distances.xaxis.axis_label = "Distance (nm)"

instability_cutoff = show(row(instability_plot,filtered_distances), notebook_handle = True)

def setCutoff(distance):
    instability_plot.renderers[-2].data_source.data['x'] = [distance,distance]
    
    cutoff_index = np.where(spec.r == distance)[0][0]
    distance_mask = spec.r*0
    distance_mask[cutoff_index:] = 1


    spec.unstable_waveform = np.dot(spec.kernel, spec.solution*distance_mask)
    spec.stable_waveform = (spec.waveform - spec.unstable_waveform)/(1.0 - spec.unstable_waveform.max())
    spec.stable_solution = spec.waveform_regularize(spec.stable_waveform)
    
    spec.stable_solution = spec.stable_solution/integrate.trapz(spec.stable_solution)
    
    filtered_distances.renderers[-1].data_source.data['y'] = spec.stable_solution
    filtered_distances.renderers[-2].data_source.data['y'] = spec.solution/integrate.trapz(spec.solution-spec.solution*distance_mask)
    
    push_notebook(handle = instability_cutoff)


#Does not currently update in real time, need to find a way to decouple with calculation of new distance distribution
instability_slider = widgets.SelectionSlider(description = 'Instability Point:', value = instability_point_start, options = spec.r,continuous_update=True)

HBox([interactive(setCutoff,distance = instability_slider)])

In [126]:
fitted

array([0, 0, 0, 0, 0])

In [132]:
model_rmsds[fitted==1]+1

array([], dtype=float64)

In [166]:
for idx in np.where(optimized ^ fitted)[0]:
    print(idx)

0
1


In [163]:
for item in np.argwhere(optimized ^ fitted):
    print(fitted[item[0]])

1
1


In [152]:
fitted ^ optimized

array([0, 0, 0, 0, 0])

In [130]:
model_rmsds[model_rmsds>0]

array([], dtype=float64)

In [175]:
#Gaussian modeling utility
ngauss = 5 # max number of gaussian populations to model
pop_models = np.arange(ngauss)
gaussian_centers = np.zeros((ngauss,ngauss)) 
gaussian_fits = np.zeros((ngauss,ngauss*3))
fitted = np.zeros(ngauss,dtype=int)
optimized = np.zeros(ngauss,dtype=int)
model_rmsds = np.zeros(ngauss)

model_plot = spec_plot(spec.r,spec.stable_solution, x_range = (1,10.1))
#initialize points on which to seed gaussian fits

for arr_row in gaussian_centers:
    model_plot.circle(arr_row,arr_row*0,visible = False)
model_plot.renderers[1].visible = True

for arr_row in gaussian_fits:
    model_plot.line(spec.r,spec.r*0, color = 'orange',visible = False)

spec.stable_fit = np.dot(spec.kernel,spec.stable_solution)
spec.stable_fit = spec.stable_fit/spec.stable_fit[0]
stable_rmsd = np.sqrt(np.mean(spec.stable_waveform-spec.stable_fit)**2)

model_rmsd_plot = spec_plot(np.linspace(0,ngauss+1),np.array([stable_rmsd*10**2]*len(np.linspace(0,ngauss+1))))
model_rmsd_plot.renderers[0].glyph.line_dash = 'dashed'
model_rmsd_plot.renderers[0].glyph.line_color = 'black'



model_rmsd_plot.line(pop_models[fitted==1]+1,model_rmsds[fitted==1], line_color = 'black', visible = False)
model_rmsd_plot.circle(pop_models[fitted==1]+1,model_rmsds[fitted==1], visible = True)
    
gaussian_modeling = show(row(model_plot,model_rmsd_plot), notebook_handle=True)

def update_gaussians(model,ith_gaussian, value):
    gaussian_centers[model,ith_gaussian] = value
    move_points(model,ith_gaussian)

def move_points(model, ith_gaussian):
    #print(tabs.selected_index)
    xs = gaussian_centers[model,:ith_gaussian+1]    
    ys = spec.stable_solution[[np.argwhere(spec.r == item)[0][0] for item in xs]]
    model_plot.renderers[model+1].data_source.data = {'x': xs, 'y': ys}
    
    push_notebook(handle = gaussian_modeling)



children = [VBox([interactive(update_gaussians, model = fixed(model), 
                              ith_gaussian = fixed(ith_gaussian), 
                              value = widgets.SelectionSlider(description = str(ith_gaussian+1)+'G', options = spec.r)) 
                  for ith_gaussian in np.arange(model+1)]) 
            for model in pop_models]

tabs = widgets.Tab()
tabs.children = children
for model in pop_models:
    tabs.set_title(model, str(model+1)+'G')

def update_figure(widget):
    #Show only points corresponding to the selected tab, only gets called once tab is clicked
    tab_idx = widget['new']
    for idx in range(1,len(model_plot.renderers)):
        model_plot.renderers[idx].visible = False
    model_plot.renderers[tab_idx+1].visible = True
    
    if fitted[tab_idx]:
        model_plot.renderers[tab_idx+1+ngauss].visible = True
    
    push_notebook(handle = gaussian_modeling)

def sum_of_gauss(x,*coeff):

    sub_coeff = np.reshape(coeff, (len(coeff)//3,3))
    
    return sum([gaussian(x,*arr_row) for arr_row in sub_coeff ])
    
def fit_gaussians(val):
    model = tabs.selected_index
    fitted[model] = 1
    optimized[model] = 0
    centers = gaussian_centers[model,:model+1]
    
    initial_con = np.array([[center,.1,.1] for center in centers])
    initial_con = initial_con.flatten()

    gaussian_fits[model,:3+model*3] = optimize.curve_fit(sum_of_gauss,spec.r,spec.stable_solution, p0 = initial_con)[0] 
    
    fit_curve = sum_of_gauss(spec.r,*gaussian_fits[model,:3+model*3])
    fit_curve = fit_curve/integrate.trapz(fit_curve)
   
    model_plot.renderers[model+1+ngauss].data_source.data = {'x': spec.r,'y': fit_curve}
    model_plot.renderers[model+1+ngauss].visible = True
    
    gaussian_waveform = np.dot(spec.kernel,fit_curve)
    gaussian_waveform = gaussian_waveform/gaussian_waveform[0]
    
    model_rmsds[model] = np.sqrt(np.mean(spec.stable_waveform-gaussian_waveform)**2)
    
    model_rmsd_plot.renderers[1].data_source.data = {'x': pop_models[fitted==1]+1 ,'y': model_rmsds[fitted==1]*10**2 }
    model_rmsd_plot.renderers[2].data_source.data = {'x': pop_models[fitted==1]+1 ,'y': model_rmsds[fitted==1]*10**2 }
    
    if np.sum(fitted)>=2:
        model_rmsd_plot.renderers[1].visible = True

    push_notebook(handle = gaussian_modeling)

def gradient_descent(initial_coeff):
    """
    intial_coeff of the form of np.array([[ngauss,3]])

    """
    
    stepsize=0.001
    mycounter=1
    dRSS=1
    lastRSS=10,
    ngauss=len(initial_coeff)
    current_coeff = initial_coeff
    
    perturbation = np.zeros((ngauss,3,ngauss,3))
    
    for i in range(ngauss):
        for j in range(3):
            perturbation[i,j,i,j] = 0.0001
    def gausswave(coeff):
        waveform = np.dot(spec.kernel,gaussians(spec.r,coeff,summation=True))
        waveform = waveform/waveform[0]
        return waveform

    while ((mycounter < 5000) & (dRSS > 10**-6)):                                      
        
        myRSS = np.linalg.norm(spec.stable_waveform-gausswave(current_coeff))**2

        dRSS= abs(lastRSS-myRSS)
        lastRSS = myRSS
        
        partialDs = np.array([[(np.linalg.norm(spec.stable_waveform-gausswave(current_coeff+perturbation[i,j]))**2 - 
                     np.linalg.norm(spec.stable_waveform-gausswave(current_coeff))**2)/.0001 for j in range(3)] for i in range(ngauss)])
        

        
        current_coeff=abs(current_coeff-stepsize*partialDs)


        mycounter+=1
    return current_coeff.reshape(ngauss*3)
    
def optimize_gaussians(val):
    
    for model in np.where(optimized ^ fitted)[0]:
    
        #model = tabs.selected_index

        initial_coeff = gaussian_fits[model,:3+model*3].reshape(model+1,3) #this is annoying, might be better of modifying gaussian_fits structure

        def gaussians(x,coeffs, summation = True):
            #coeffs in form of [[ngauss,3]] and component order r,s,a
            gaussians = (coeffs[:,2]*np.exp(-((x[:,np.newaxis]-coeffs[:,0])**2/(2*coeffs[:,1]**2)))).T

            if summation:
                return np.sum(gaussians,axis = 0)
            else:
                return gaussians

        gaussian_fits[model,:3+model*3] = gradient_descent(initial_coeff)

        fit_curve = gaussians(spec.r,gaussian_fits[model,:3+model*3].reshape(model+1,3))
        fit_curve = fit_curve/integrate.trapz(fit_curve)
        
        for idx in range(1,len(model_plot.renderers)):
            model_plot.renderers[idx].visible = False
        
        model_plot.renderers[model+1+ngauss].data_source.data = {'x': spec.r,'y': fit_curve}
        model_plot.renderers[model+1+ngauss].visible = True
        
        gaussian_waveform = np.dot(spec.kernel,fit_curve)
        gaussian_waveform = gaussian_waveform/gaussian_waveform[0]

        model_rmsds[model] = np.sqrt(np.mean(spec.stable_waveform-gaussian_waveform)**2)

        model_rmsd_plot.renderers[1].data_source.data = {'x': pop_models[fitted==1]+1 ,'y': model_rmsds[fitted==1]*10**2 }
        model_rmsd_plot.renderers[2].data_source.data = {'x': pop_models[fitted==1]+1 ,'y': model_rmsds[fitted==1]*10**2 }

        optimized[model] = 1

        push_notebook(handle = gaussian_modeling)
    

tabs.observe(update_figure, names = 'selected_index')

fit_gauss_button=widgets.Button(description='Fit Gaussian(s)')
optimize_gauss_button=widgets.Button(description='Optimize')

fit_gauss_button.on_click(fit_gaussians)
optimize_gauss_button.on_click(optimize_gaussians)

HBox([tabs,VBox([fit_gauss_button,optimize_gauss_button])])



In [174]:
gaussian_fits

array([[4.39776994, 0.49225491, 0.01458946, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [3.50718104, 0.29970442, 0.06055791, 4.51895949, 0.37562434,
        0.25231437, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 

In [173]:
gaussian_fits

array([[4.46424713e+00, 4.35878353e-01, 1.45894621e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.53471989e+00, 3.29563210e-01, 3.33602776e-03, 4.50867702e+00,
        3.59207762e-01, 1.55134077e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+0

In [26]:
def gaussians(x,coeffs, summation = True):
    #coeffs in form of [[ngauss,3]] and component order r,s,a
    gaussians = (coeffs[:,2]*np.exp(-((x[:,np.newaxis]-coeffs[:,0])**2/(2*coeffs[:,1]**2)))).T
    
    if summation:
        return np.sum(gaussians,axis = 0)
    else:
        return gaussians



In [9]:
def gaussian(x,r,s,a):
    return a*np.exp(-((x-r)**2/(2*s**2)))

In [20]:
 def gausswave(coeff):
        waveform = np.dot(spec.kernel,gaussians(spec.r,coeff,summation=True))
        waveform = waveform/waveform[0]
        return waveform

In [70]:
#https://stackoverflow.com/questions/30184926/bokeh-how-to-click-and-drag
model_plot = spec_plot(spec.r,spec.stable_solution)

template="""
<div style="background: white; color: black">
<%= value %></div>
"""

formater =  HTMLTemplateFormatter(template=template)

source = ColumnDataSource({
    'x': [1], 'y': [1], 'color': ['black']
})

renderer = model_plot.scatter(x='x', y='y', source=source, color='color', size=10)
columns = [TableColumn(field="x", title="x", formatter = formater),
           TableColumn(field="y", title="y",formatter = formater),
           TableColumn(field='color', title='color',formatter = formater)]
table = DataTable(source=source, columns=columns, editable=True, height=900, width  = 300, index_header = '1G')

draw_tool = PointDrawTool(renderers=[renderer], empty_value='black')
model_plot.add_tools(draw_tool)
model_plot.toolbar.active_tap = draw_tool




show(row(table,model_plot))

In [68]:
model_plot.renderers

[GlyphRenderer(id='6700', ...), GlyphRenderer(id='6707', ...)]

In [22]:
import bokeh
bokeh.__version(__

'2.1.0'

In [39]:
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter
from bokeh.io import show


dict1 = {'x':[0]*6, 
         'y':[0, 1, 0, 1, 0, 1],
         'color':['gray', 'red', 'blue', 'red', 'blue', 'red']}
source = ColumnDataSource(data=dict1)

template="""
<div style="background: white; color: black">
<%= value %></div>
"""

formater =  HTMLTemplateFormatter(template=template)


columns = [
    TableColumn(field="x", title="x",formatter = formater),
    TableColumn(field="y", title="y",formatter = formater)
]

data_table = DataTable(source=source, columns=columns, width=200)

In [40]:
show(data_table)

In [36]:
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter
from bokeh.io import show

dict1 = {'x':[0]*6,'y':[0,1,0,1,0,1]}
source = ColumnDataSource(data=dict1)

template="""
<div style="background: white; 
    color: black"> 
<%= value %></div>
"""

template = """
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>{{ title|e if title else "Bokeh Plot" }}</title>
        {{ bokeh_css }}
        {{ bokeh_js }}
        <style>
          .slick-header-column {
            background-color: lightgray !important;
            background-image: none !important;
            color: black;
          }
        </style>
    </head>
    <body>
        {{ plot_div|indent(8) }}
        {{ plot_script|indent(8) }}
    </body>
</html>
"""

formater =  HTMLTemplateFormatter(template=template)
columns = [
    TableColumn(field="x", title="x",formatter = formater),
    TableColumn(field="y", title="y",formatter=formater)
]

data_table = DataTable(source=source, columns=columns, width=800, css_classes=["mycustom"])

show(data_table)

In [54]:
from bokeh.plotting import figure, output_notebook, show, Column
from bokeh.models import DataTable, TableColumn, PointDrawTool, ColumnDataSource


def modify_doc(doc): 
    p = figure(x_range=(0, 10), y_range=(0, 10), tools=[],
           title='Point Draw Tool')
    p.background_fill_color = 'lightgrey'
    source = ColumnDataSource({
        'x': [1, 5, 9], 'y': [1, 5, 9], 'color': ['red', 'green', 'yellow']
    })
    renderer = p.scatter(x='x', y='y', source=source, color='color', size=10)
    columns = [TableColumn(field="x", title="x"),
               TableColumn(field="y", title="y"),
               TableColumn(field='color', title='color')]
    table = DataTable(source=source, columns=columns, editable=True, height=200)

    draw_tool = PointDrawTool(renderers=[renderer], empty_value='black')
    p.add_tools(draw_tool)
    p.toolbar.active_tap = draw_tool

    doc.add_root(Column(p, table))

show(modify_doc)

In [52]:
draw_tool.

{'list1': [0, 1, 2, 3], 'list2': [4, 5, 6, 7]}

In [254]:
spec.kernel.shape

(685, 512)